In [105]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

In [106]:
data_folder = './data/'
pickle_folder = data_folder + 'pickle/'
imdb_folder = data_folder + 'IMDB/'

In [107]:
title_basics = pd.read_csv(imdb_folder + 'title.basics.tsv', sep='\t', low_memory=False)
title_ratings = pd.read_csv(imdb_folder + 'title.ratings.tsv', sep='\t', low_memory=False)

display(title_basics.sample(10))
display(title_ratings.sample(10))

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1242467,tt10496906,tvEpisode,Placenta Capillaire,Placenta Capillaire,0,2017,\N,\N,Talk-Show
1987553,tt11838360,tvEpisode,Episode #1.2354,Episode #1.2354,0,\N,\N,\N,Crime
5811079,tt22804582,tvSeries,Superheltskolen,Superheltskolen,0,2022,\N,\N,Comedy
9110550,tt5485148,tvEpisode,Episode #1.882,Episode #1.882,0,1959,\N,30,"Crime,Drama,Mystery"
7903464,tt32769146,tvEpisode,Episode #1.1288,Episode #1.1288,0,1977,\N,30,News
10903766,tt9451726,tvEpisode,Episode #1.541,Episode #1.541,0,2010,\N,\N,Drama
5039004,tt1954889,tvMovie,The Bill of Rights,The Bill of Rights,0,2010,\N,\N,History
5147106,tt2011202,short,Regreso a Viridiana,Regreso a Viridiana,0,2011,\N,25,"Documentary,Short"
7915347,tt32812998,tvEpisode,Episode #1.3609,Episode #1.3609,0,1986,\N,30,News
4151002,tt15847384,tvEpisode,Episode #1.7,Episode #1.7,0,2005,\N,\N,Comedy


,tconst,averageRating,numVotes
704712,tt14427564,9.2,17
747839,tt1529790,6.6,7
49351,tt0070947,7.5,4261
1135265,tt3495220,4.4,13
561077,tt1168969,7.7,253
1280994,tt5756058,6.8,178
862221,tt19372822,9.0,9
1029353,tt27808548,7.5,10
353053,tt0678751,8.1,23
512648,tt10851586,8.2,13


In [128]:
with open(pickle_folder + 'movies_date.p', 'rb') as f:
    movies = pickle.load(f)

In [116]:
# filter to include only movies et cree les colonnes
title_basics_movies = title_basics[title_basics['titleType'] == 'movie'][['tconst', 'primaryTitle', 'startYear']]
title_basics_movies['startYear'] = pd.to_numeric(title_basics_movies['startYear'], errors='coerce').fillna(0).astype(int)
title_basics_movies = title_basics_movies.dropna(subset=['primaryTitle', 'startYear'])

In [117]:
imdb_data = title_basics_movies.merge(title_ratings, on='tconst', how='left')

In [129]:
movies['Movie_box_office_revenue'] = pd.to_numeric(movies['Movie_box_office_revenue'], errors='coerce')
movies = movies.dropna(subset=['Movie_name', 'Year'])

# Merge IMDb and CMU Data
merged_data = movies.merge(
    imdb_data[['primaryTitle', 'startYear', 'averageRating', 'numVotes']],
    left_on=['Movie_name', 'Year'],
    right_on=['primaryTitle', 'startYear'],
    how='left'
).drop(columns=['primaryTitle', 'startYear'])

merged_data

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages_(Freebase_ID:name_tuples),Movie_countries_(Freebase_ID:name_tuples),Movie_genres_(Freebase_ID:name_tuples),Year,Year Interval,Year_Interval,averageRating,numVotes
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001,1990-2016,1990-2016,4.9,58585.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000,1990-2016,1990-2016,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988,1970-1990,1970-1990,5.6,42.0
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987,1970-1990,1970-1990,NaN,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983,1970-1990,1970-1990,5.9,648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74980,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011,1990-2016,1990-2016,NaN,NaN
74981,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011,1990-2016,1990-2016,6.8,3249.0
74982,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972,1970-1990,1970-1990,5.9,117.0
74983,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992,1990-2016,1990-2016,NaN,NaN


In [124]:
films_with_ratings = merged_data['averageRating'].notna().sum()
print(f"Nb of films with IMDb ratings: {films_with_ratings}")
films_without_ratings = merged_data['averageRating'].isna().sum()
print(f"Nb of films without IMDb ratings: {films_without_ratings}")

merged_data = merged_data.dropna(subset=['averageRating','numVotes'])

Nb of films with IMDb ratings: 40197
Nb of films without IMDb ratings: 34835


In [130]:
pickle.dump( merged_data, open(pickle_folder + "movies_averageRating.p", "wb" ) )

In [131]:
with open(pickle_folder + 'movies_countries_exploded.p', 'rb') as f:
    movies_a = pickle.load(f)
movies_b = pd.merge(merged_data[['Wikipedia_movie_ID','averageRating','numVotes','Year','Year_Interval']],movies_a, on='Wikipedia_movie_ID', how='left')
pickle.dump(movies_b, open(pickle_folder + "movies_countries_ratings.p", "wb" ) )

with open(pickle_folder + 'movies_languages_exploded.p', 'rb') as f:
    movies_a = pickle.load(f)
movies_b = pd.merge(merged_data[['Wikipedia_movie_ID','averageRating','numVotes','Year','Year_Interval']],movies_a, on='Wikipedia_movie_ID', how='left')
pickle.dump(movies_b, open(pickle_folder + "movies_languages_exploded_ratings.p", "wb" ) )

with open(pickle_folder + 'movies_season.p', 'rb') as f:
    movies_a = pickle.load(f)
movies_b = pd.merge(merged_data[['Wikipedia_movie_ID','averageRating','numVotes','Year','Year_Interval']],movies_a, on='Wikipedia_movie_ID', how='left')
pickle.dump(movies_b, open(pickle_folder + "movies_season_ratings.p", "wb" ) )

with open(pickle_folder + 'movies_genres_exploded.p', 'rb') as f:
    movies_a = pickle.load(f)
movies_b = pd.merge(merged_data[['Wikipedia_movie_ID','averageRating','numVotes','Year','Year_Interval']],movies_a, on='Wikipedia_movie_ID', how='left')
pickle.dump(movies_b, open(pickle_folder + "movies_genres_ratings.p", "wb" ) )